In [ ]:

from google.colab import drive
drive.mount('/content/drive')

**source:** https://www.kaggle.com/awalahmedfime/bacteria-segmentation/edit/run/42535923

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

**Constants**

In [ ]:
np.random.seed(41)
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 16
NUM_CLASSES = 3
IMG_PATH = '/content/drive/My Drive/Machine Learning/iccv09 3 class/train Image/'
MASK_PATH = '/content/drive/My Drive/Machine Learning/iccv09 3 class/mask/'
#LABEL_PATH = '../input/bacteria-detection-with-darkfield-microscopy/masks/'
IMG_SUB_PATH = '/content/drive/My Drive/Machine Learning/iccv09 3 class/train Image/Image/'
MASK_SUB_PATH = '/content/drive/My Drive/Machine Learning/iccv09 3 class/train One hot/One hot/'
ONEHOT_MASK = '/content/drive/My Drive/Machine Learning/iccv09 3 class/mask/mask'

Mask need to be onehot incoded

In [ ]:
test_p = "/content/drive/My Drive/Machine Learning/iccv09 3 class/test Image/"
test_m = "/content/drive/My Drive/Machine Learning/iccv09 3 class/test one hot/"


tst_fies = os.listdir(test_m)
tst_fies.sort()
test= np.zeros((len(tst_fies), IMAGE_HEIGHT , IMAGE_WIDTH, 3), dtype= np.bool)
for n, mf in tqdm(enumerate(tst_fies), total=len(tst_fies)):
  img = cv2.imread(os.path.join(test_m, mf) )
  img = cv2.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
  img = img / 255
  test[n] = img

In [ ]:
from keras import backend as K

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def jaccard_index(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    #return (1 - jac) * smooth
    return jac

def jaccard_index_loss(y_true, y_pred, smooth=1):
    return (1 - jaccard_index(y_true, y_pred)) * smooth

In [ ]:
pip install scikit-plot

In [ ]:
modelFile = "/content/drive/MyDrive/Machine Learning/iccv09 3 class/resnet101_dice_coef_loss_Yes.h5"
#model = keras.models.load_model(modelFile)
#model = keras.models.load_model(modelFile, custom_objects={'jaccard_index_loss': jaccard_index_loss, 'jaccard_index':jaccard_index})
model = keras.models.load_model(modelFile, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
#Reloading model
#model = keras.models.load_model("/content/drive/MyDrive/Machine Learning/iccv09 3 class/Weight1/resnet50 dice_coef_loss with weight 41/resnet50_with_wights.h5")

In [ ]:

test_files = os.listdir(test_p)
test_files.sort()
y_score= np.zeros((len(tst_fies), IMAGE_HEIGHT , IMAGE_WIDTH, 3), dtype= np.float)
for n, mf in tqdm(enumerate(test_files), total=len(test_files)):
  img = cv2.imread(os.path.join(test_p, mf) )
  img =  cv2.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
  img=np.expand_dims(img, 0)
  img = model.predict(img)
  img =  np.squeeze(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  y_score[n]=img

y_true = test*1
y_probas = y_score*1

In [ ]:
cls1=np.zeros((100, 256, 256), dtype= np.bool)
for i in range(100):
  for j in range(256):
    for k in range(256):
      tr=np.argmax(y_true[i,j,k,:])
      pr=np.argmax(y_probas[i,j,k,:])
      cls1[i][j][k]=(tr==pr)

cls1=cls1.reshape(-1)

In [ ]:
modelFile = "/content/drive/MyDrive/Machine Learning/iccv09 3 class/resnet50_categorical_crossentropy_loss_Yes.h5"
model1 = keras.models.load_model(modelFile)
#model1 = keras.models.load_model(modelFile, custom_objects={'jaccard_index_loss': jaccard_index_loss})
#model1 = keras.models.load_model(modelFile, custom_objects={'dice_coef_loss': dice_coef_loss})
#model1 = keras.models.load_model(modelFile, custom_objects={'jaccard_index_loss': jaccard_index_loss, 'jaccard_index':jaccard_index})
#model1 = keras.models.load_model(modelFile, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
#Reloading model
#model = keras.models.load_model("/content/drive/MyDrive/Machine Learning/iccv09 3 class/Weight1/resnet50 dice_coef_loss with weight 41/resnet50_with_wights.h5")

In [ ]:

test_files = os.listdir(test_p)
test_files.sort()
y_score1= np.zeros((len(tst_fies), IMAGE_HEIGHT , IMAGE_WIDTH, 3), dtype= np.float)
for n, mf in tqdm(enumerate(test_files), total=len(test_files)):
  img = cv2.imread(os.path.join(test_p, mf) )
  img =  cv2.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
  img=np.expand_dims(img, 0)
  img = model1.predict(img)
  img =  np.squeeze(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  y_score1[n]=img

y_probas1 = y_score1*1

In [ ]:
cls2=np.zeros((len(tst_fies), 256, 256), dtype= np.bool)
for i in range(len(tst_fies)):
  for j in range(256):
    for k in range(256):
      tr=np.argmax(y_true[i,j,k,:])
      pr=np.argmax(y_probas1[i,j,k,:])
      cls2[i][j][k]=(tr==pr)

cls2=cls2.reshape(-1)

In [ ]:
cls2.shape

In [ ]:
cont=np.zeros((2,2), dtype= np.int)
for i in range(len(cls2)):
  cont[1-cls1[i]][1-cls2[i]]=cont[1-cls1[i]][1-cls2[i]]+1

cont

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(cont, exact=True)
# summarize the finding
print('statistic=%f, p-value=%f' % (result.statistic, result.pvalue))

In [ ]:
result = mcnemar(cont, exact=False)
# summarize the finding
print('statistic=%f, p-value=%f' % (result.statistic, result.pvalue))